# Vacation Py
## Using API to make maps of cities from WeatherPy and find Hotels

In [1]:
# Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Make DataFrame from list of cities from WeatherPy

In [9]:
#pull cities from the csv file from WeatherPy and put into a DataFrame
city_data = pd.read_csv("../Output_Data/Cities.csv")
city_data = city_data.drop("Unnamed: 0", axis=1)
city_data.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.65,115.33,93.99,9,4,5.68,AU,1607657665
1,Tuktoyaktuk,69.45,-133.04,-9.40,76,75,11.41,CA,1607657665
2,Mar del Plata,-38.00,-57.56,73.99,69,0,11.41,AR,1607657429
3,Punta Arenas,-53.15,-70.92,44.60,81,0,5.82,CL,1607657665
4,Ushuaia,-54.80,-68.30,50.00,76,75,4.92,AR,1607657666


In [10]:
#define variables for map
cities = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"].astype(float)

### Humidity Heatmap

In [11]:
#map dataframe with heatmap using the humidity
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities, weights = humidity, dissipating=False, max_intensity =100, point_radius=3)
gmaps.figure()
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
#which cities have "ideal weather"
city_data[(city_data["Max Temp"] >= 70) & (city_data["Max Temp"] < 80) & 
                          (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,New Norfolk,-42.78,147.06,72.00,35,0,1.99,AU,1607657669
75,Boyuibe,-20.42,-63.28,70.63,62,0,7.52,BO,1607657678
180,Puerto El Triunfo,13.28,-88.55,73.40,78,0,4.70,SV,1607657695
213,Vryburg,-26.96,24.73,70.27,33,0,5.93,ZA,1607657701
241,Cravo Norte,6.30,-70.20,74.66,94,0,5.26,CO,1607657709
274,Young,-32.68,-57.63,72.00,45,0,9.93,UY,1607657716
371,Padrauna,26.90,83.98,71.08,47,0,3.04,IN,1607657732
426,Rayachoti,14.05,78.75,73.40,88,0,5.21,IN,1607657742
482,Tura,25.52,90.22,73.26,51,0,0.67,IN,1607657752


## Ideal Weather DataFrame

In [13]:
#take those results and turn into a dataframe
ideal_weather = city_data.loc[(city_data["Max Temp"] >= 70) & (city_data["Max Temp"] < 80) & 
                           (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]
ideal_weather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,New Norfolk,-42.78,147.06,72.00,35,0,1.99,AU,1607657669
75,Boyuibe,-20.42,-63.28,70.63,62,0,7.52,BO,1607657678
180,Puerto El Triunfo,13.28,-88.55,73.40,78,0,4.70,SV,1607657695
213,Vryburg,-26.96,24.73,70.27,33,0,5.93,ZA,1607657701
241,Cravo Norte,6.30,-70.20,74.66,94,0,5.26,CO,1607657709
274,Young,-32.68,-57.63,72.00,45,0,9.93,UY,1607657716
371,Padrauna,26.90,83.98,71.08,47,0,3.04,IN,1607657732
426,Rayachoti,14.05,78.75,73.40,88,0,5.21,IN,1607657742
482,Tura,25.52,90.22,73.26,51,0,0.67,IN,1607657752


#### There are 9 cities with "ideal weather" of being between 70 and 80 degrees (F), with wind speed less than 10 mph, and zero cloudiness.

## Finding Hotels for cities with Ideal Weather

In [14]:
#API Reuqest for hotel information
#variables for url
radius = "5000"
place_type = "lodging"
hotel_data = []
#text to print
print("Beginning Data Retrieval")
print("----------------------------")

#for loop for the cities in the ideal weather dataframe
for index, city in ideal_weather.iterrows():
    city_name = city["City"]
    country = city["Country"]
    #variables for url
    latitude = city["Latitude"]
    longitude = city["Longitude"]
    location = f"{latitude},{longitude}"
    #base url
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={place_type}&key={g_key}"
    response = requests.get(url).json()
    
    #use the try function in case hotel not found
    try:
        #store hotel information        
        hotel_name = response["results"][0]["name"]
        hotel_dict = {"Hotel Name": hotel_name, "City":city_name, "Country":country, "Lat":latitude, "Lng":longitude}
        hotel_data.append(hotel_dict)
        #print result for name
        print(f"Processing hotel: {hotel_name}")
    
    #for any cities without a hotel in range    
    except IndexError:
        print("No hotel found")
        continue
#print updated dataframe        
#print(hotel_data)
print("----------------------------")
print("Data Retrieval Complete")
print("----------------------------")

Beginning Data Retrieval
----------------------------
Processing hotel: The Shingles Riverside Cottages
Processing hotel: Residencial Eluzai
Processing hotel: Hotel Jardin
Processing hotel: Times Premier Lodge
Processing hotel: Hotel El Puerto
Processing hotel: Posada Gira el Sol
Processing hotel: Hotel Sky Lark
Processing hotel: Swetha lodge
Processing hotel: Hotel RIKMAN Continental
----------------------------
Data Retrieval Complete
----------------------------


#### All 9 cities have hotels within range

## Hotel DataFrame

In [15]:
#making data frame for hotel
hotel_df = pd.DataFrame(hotel_data)
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,The Shingles Riverside Cottages,New Norfolk,AU,-42.78,147.06
1,Residencial Eluzai,Boyuibe,BO,-20.42,-63.28
2,Hotel Jardin,Puerto El Triunfo,SV,13.28,-88.55
3,Times Premier Lodge,Vryburg,ZA,-26.96,24.73
4,Hotel El Puerto,Cravo Norte,CO,6.30,-70.20
5,Posada Gira el Sol,Young,UY,-32.68,-57.63
6,Hotel Sky Lark,Padrauna,IN,26.90,83.98
7,Swetha lodge,Rayachoti,IN,14.05,78.75
8,Hotel RIKMAN Continental,Tura,IN,25.52,90.22


## Hotel Map

In [16]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
#mappling the previous map again but now with the hotel information for some of the cities
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities, weights = humidity, dissipating=False, max_intensity =100, point_radius=2)
fig.add_layer(heat_layer)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

Previous heat map with the information for the hotels in the ideal weather cities. We can see all 9 cities along with their hotel information. 